In [1]:
import pandas as pd
import operator

In [2]:
import json
import math
import requests

In [3]:
with open('data.json') as json_data:
    data = json.load(json_data)

## Clean data

In [4]:
cleaned_data = {}
for player in data:
    entry = {}
    for hero in data[player]:
        if not math.isnan(data[player][hero]):
            entry[hero] = data[player][hero]
    if len(entry) != 0:
        cleaned_data[player] = entry

In [5]:
with open('cleaned_data.json', 'w') as outfile:
    json.dump(cleaned_data, outfile)

In [6]:
df = pd.DataFrame(cleaned_data)
df.head()

,88018,88470,145875,514364,690740,758174,1185644,1296625,1522705,2553754,...,372273611,372595310,375600147,376376587,380320771,382225911,385750274,389033587,393424689,397462905
1,2.722944,0.937091,100.0,0.721559,1.020278,0.240423,0.692306,0.597255,0.413706,0.323357,...,NaN,0.352596,NaN,1.638541,0.309405,3.982976,2.954567,1.098722,0.297777,0.163170
10,1.166794,1.052196,NaN,NaN,0.216978,0.520793,2.306300,0.791051,0.446418,0.118150,...,NaN,1.470582,NaN,5.313786,0.364983,3.213039,1.625012,2.844958,NaN,1.332554
100,NaN,0.141753,NaN,3.710877,0.726096,8.861626,1.173608,0.643014,0.520907,0.910191,...,NaN,0.334966,NaN,0.047741,2.025888,0.152128,NaN,0.096138,0.595554,0.163170
101,0.050467,0.165379,NaN,3.006498,0.048217,0.483594,0.820510,2.221675,0.842734,1.365286,...,NaN,0.000000,NaN,0.190964,0.399034,NaN,NaN,0.040058,0.148889,0.163170
102,NaN,0.219826,0.0,NaN,1.767193,1.555057,0.266666,0.722147,1.117931,0.801158,...,2.360225,0.258461,NaN,0.127309,0.476713,0.202837,0.000000,0.080115,0.000000,1.101397


## Recommender Sys

In [7]:
with open('cleaned_data.json') as json_data:
    data = json.load(json_data)
    
with open('hero_name.json') as json_data:
    hero_name = json.load(json_data)

In [8]:
def pearson_similarity(person1, person2):

    common_ranked_items = [itm for itm in data[person1] if itm in data[person2]]

    n = len(common_ranked_items)

    s1 = sum([data[person1][item] for item in common_ranked_items])
    s2 = sum([data[person2][item] for item in common_ranked_items])

    ss1 = sum([pow(data[person1][item], 2) for item in common_ranked_items])
    ss2 = sum([pow(data[person2][item], 2) for item in common_ranked_items])

    ps = sum([data[person1][item] * data[person2][item] for item in common_ranked_items])

    num = n * ps - (s1 * s2)

    den = math.sqrt((n * ss1 - math.pow(s1, 2)) * (n * ss2 - math.pow(s2, 2)))

    return (num / den) if den != 0 else 0


def recommend(person, bound, similarity=pearson_similarity):
    scores = [(similarity(person, other), other) for other in data if other != person]

    scores.sort()
    scores.reverse()
    scores = scores[0:bound]
    
    recomms = {}

    for sim, other in scores:
        ranked = data[other]

        for itm in ranked:
            if itm not in data[person]:
                weight = sim * ranked[itm]

                if itm in recomms:
                    s, weights = recomms[itm]
                    recomms[itm] = (s + sim, weights + [weight])
                else:
                    recomms[itm] = (sim, [weight])

    for r in recomms:
        sim, item = recomms[r]
        recomms[r] = sum(item) / sim

    return recomms, scores

In [9]:
MY_API_HEROES = ''
OPEN_DOTA_PLAYER_HEROES = 'https://api.opendota.com/api/players/{}/heroes/?api_key=' + MY_API_HEROES

def get_player_stats_response(player_id):
    '''Get player stats request'''
    try:
        response = requests.get(OPEN_DOTA_PLAYER_HEROES.format(player_id))
        data_json = json.loads(response.content.decode('utf-8'))
    except:
        data_json = None
    return data_json


# Algorithm to compute score
def compute_score(data, player_id):
    total_scores = 0.0
    hero_data = {}
    
    total_score = 0.0
    for entry in data:
        if entry['games'] != 0:
            hero_id = entry['hero_id']
            score = entry['win'] * entry['win'] / entry['games']
            total_score += score
            
            hero_data[hero_id] = score
    
    for key, value in hero_data.items():
        normalized_score = 100.0 * value / total_score
        hero_data[key] = normalized_score
    
    return hero_data


def recommend_user(player_id):
    user_hero_stats = get_player_stats_response(player_id)
    hero_data = compute_score(user_hero_stats, player_id)
    
    data[str(user_id)] = hero_data
    
    results, scores = recommend(str(player_id), 10)
    
    del data[str(user_id)]

    sorted_hero_data = sorted(hero_data.items(), key=operator.itemgetter(1), reverse=True)[0:3]
    
    message = '''
        Your best heroes are: {}, {}, {}
        Your play style is similar to:
            https://www.dotabuff.com/players/{}
            https://www.dotabuff.com/players/{}
            https://www.dotabuff.com/players/{}
        We recommend you to practice:
        '''.format(
            hero_name[str(sorted_hero_data[0][0])],
            hero_name[str(sorted_hero_data[1][0])],
            hero_name[str(sorted_hero_data[2][0])],
            scores[0][1],
            scores[1][1],
            scores[2][1]
            )
    
    message += '\t'
    for rec_hero_id in list(results)[0:3]:
        message += (str(hero_name[rec_hero_id]) + ', ')
    message += '\n'
    
    print(message)

In [11]:
user_id = 112463413
recommend_user(user_id)


        Your best heroes are: Luna, Disruptor, Riki
        Your play style is similar to:
            https://www.dotabuff.com/players/5390881
            https://www.dotabuff.com/players/89296893
            https://www.dotabuff.com/players/106285240
        We recommend you to practice:
        	Lifestealer, Lina, Clinkz, 

